In [4]:
%pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 6.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


try:
    response = requests.get("https://coinmarketcap.com/")
    soup = BeautifulSoup(response.text, 'html.parser')

    table_body = soup.find('table', class_="sc-db1da501-3 ccGPRR cmc-table").find("tbody")
    rows = table_body.find_all('tr')
    coins = {"rank": [], "name" : [], "price" : [], "market_cap" : [], "volume" : [], "circulating_supply" : []};

    count = 0  # Counter to limit to first 10

    for row in rows:
        cols = row.find_all('td')
        if len(cols) < 10:
            continue  # Skip malformed rows

        try:
            # Rank
            rank = cols[1].text.strip()

            # Name
            name_tag = cols[2].find('p', class_='coin-item-name')
            name = name_tag.text.strip() if name_tag else "N/A"

            # Price
            price_tag = cols[3].find('span')
            price = price_tag.text.strip() if price_tag else "N/A"

            # Market Cap
            market_cap_tag = cols[7].find('span', class_='sc-11478e5d-0')
            market_cap = market_cap_tag.text.strip() if market_cap_tag else "N/A"

            # Volume (24h)
            volume_tag = cols[8].find('p')
            volume = volume_tag.text.strip() if volume_tag else "N/A"

            # Circulating Supply
            supply_tag = cols[9].find('div', class_='circulating-supply-value')
            circulating_supply = supply_tag.text.strip() if supply_tag else "N/A"

            # Final Output
            # print(rank, name, price, market_cap, volume, circulating_supply)
            coins["rank"].append(rank)
            coins["name"].append(name_tag)
            coins["price"].append(price_tag)
            coins["market_cap"].append(market_cap_tag)
            coins["volume"].append(volume_tag)
            coins["circulating_supply"].append(supply_tag)

            count += 1
            if count == 10:
                break  # Stop after first 10 valid rows

        except Exception as e:
            print(f"Skipping row due to error: {e}")

except Exception as e:
    print("Error:", e)


df = pd.DataFrame(data=coins)

In [5]:
print(df)

  rank        name         price  market_cap             volume  \
0    1   [Bitcoin]  [$94,949.28]    [$1.89T]  [$25,834,759,358]   
1    2  [Ethereum]   [$1,807.40]  [$218.34B]  [$14,664,838,700]   
2    3    [Tether]       [$1.00]  [$148.46B]  [$57,884,294,372]   
3    4       [XRP]       [$2.25]  [$131.53B]   [$2,818,706,241]   
4    5       [BNB]     [$604.27]   [$85.14B]   [$1,593,054,872]   
5    6    [Solana]     [$147.79]   [$76.51B]   [$2,819,066,729]   
6    7      [USDC]       [$1.00]   [$62.02B]   [$8,805,004,026]   
7    8  [Dogecoin]     [$0.1757]   [$26.19B]     [$894,281,500]   
8    9   [Cardano]     [$0.6996]    [$24.7B]     [$815,075,219]   
9   10      [TRON]     [$0.2434]   [$23.11B]     [$450,501,370]   

        circulating_supply  
0    [[19.85M],  ,  , BTC]  
1   [[120.73M],  ,  , ETH]  
2  [[148.41B],  ,  , USDT]  
3    [[58.44B],  ,  , XRP]  
4   [[140.89M],  ,  , BNB]  
5   [[517.66M],  ,  , SOL]  
6   [[62.02B],  ,  , USDC]  
7  [[149.06B],  ,  , DOGE]  
8